## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [8]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"


input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''
model_xcp = keras.applications.Xception(include_top=False, weights='imagenet', input_tensor=input_tensor, pooling=None,
                                        classes=10)
'''Resnet 50 架構'''
model_res = keras.applications.ResNet50(include_top=False, weights='imagenet',
                                        input_tensor=input_tensor,
                                        pooling=None, classes=10)
model_xcp.summary()

C:\Users\brian\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 15, 15, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 15, 15, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 15, 15, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [9]:
model_res.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

## 添加層數

In [10]:
x_xcp = model_xcp.output
x_res = model_res.output

'''可以參考Cifar10實作章節'''
x_xcp = GlobalAveragePooling2D()(x_xcp)
x_xcp = Dense(output_dim=128, activation='relu')(x_xcp)
x_xcp = Dropout(p=0.1)(x_xcp)
predictions_xcp = Dense(output_dim=10,activation='softmax')(x_xcp)
model_xcp = Model(inputs=model_xcp.input, outputs=predictions_xcp)
print('Model Xception 深度：', len(model_xcp.layers))

x_res = GlobalAveragePooling2D()(x_res)
x_res = Dense(output_dim=128, activation='relu')(x_res)
x_res = Dropout(p=0.1)(x_res)
predictions_res = Dense(output_dim=10,activation='softmax')(x_res)
model_res = Model(inputs=model_res.input, outputs=predictions_res)
print('Model Resnet50 深度：', len(model_res.layers))

Model Xception 深度： 136
Model Resnet50 深度： 179


C:\Users\brian\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  
C:\Users\brian\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  import sys
C:\Users\brian\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
  
C:\Users\brian\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  del sys.path[0]
C:\Users\brian\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  
C:\Users\brian\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
 

In [11]:
Models = [model_xcp, model_res]

## 鎖定特定幾層不要更新權重

In [12]:
for model in Models:
    for layer in model.layers[:30]:
        layer.trainable = False
    for layer in model.layers[30:]:
        layer.trainable = True

## 準備 Cifar 10 資料

In [13]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0,1,2,3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

(50000, 32, 32, 3)


C:\Users\brian\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


## Training

In [14]:
# compile the model (should be done *after* setting layers to non-trainable)
for model in Models:
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    model.fit(x_train,y_train,batch_size=32,epochs=30, validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 107s 2ms/step - loss: 1.1993 - acc: 0.5918 - val_loss: 2.2760 - val_acc: 0.3986cc: 0. - ETA: 1s - loss: 1.2019 - acc: - ETA: 0s - loss: 1.2011 - ac
Epoch 2/30
50000/50000 [==============================] - 98s 2ms/step - loss: 0.7684 - acc: 0.7468 - val_loss: 2.1504 - val_acc: 0.4431
Epoch 3/30
50000/50000 [==============================] - 92s 2ms/step - loss: 0.6111 - acc: 0.7984 - val_loss: 1.8751 - val_acc: 0.5116 - loss: 
Epoch 4/30
50000/50000 [==============================] - 97s 2ms/step - loss: 0.4989 - acc: 0.8332 - val_loss: 1.9963 - val_acc: 0.5331
Epoch 5/30
50000/50000 [==============================] - 97s 2ms/step - loss: 0.3980 - acc: 0.8693 - val_loss: 1.7568 - val_acc: 0.5534
Epoch 6/30
50000/50000 [==============================] - 97s 2ms/step - loss: 0.3170 - acc: 0.8940 - val_loss: 2.4201 - val_acc: 0.4845
Epoch 7/30
50000/50000 [==========================

50000/50000 [==============================] - 127s 3ms/step - loss: 0.1348 - acc: 0.9586 - val_loss: 6.7589 - val_acc: 0.1002
Epoch 28/30
50000/50000 [==============================] - 127s 3ms/step - loss: 0.1074 - acc: 0.9663 - val_loss: 5.0513 - val_acc: 0.1003
Epoch 29/30
50000/50000 [==============================] - 127s 3ms/step - loss: 0.1077 - acc: 0.9664 - val_loss: 6.2617 - val_acc: 0.1000
Epoch 30/30
50000/50000 [==============================] - 127s 3ms/step - loss: 0.1016 - acc: 0.9679 - val_loss: 10.2990 - val_acc: 0.1000
